# Data analyzing of the 360-degree videos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import glob
from datetime import datetime as datetime
import time

In [ ]:
# Configure panda to show all columns
pd.set_option('display.max_columns', None)

### Merging all the traces found

In [ ]:
path = './traces' # use your path
all_files = glob.glob(path + "/*.json")

li = []

for filename in all_files:
    df = pd.read_json(filename)
    li.append(df)

frame = pd.concat(li, ignore_index=True)

og_frame = pd.concat(li, ignore_index=True)

In [ ]:
# Convert upload date (yyymmdd) to "days since upload"

current_time = datetime.now().timestamp()

for i in frame.index:
    try:
        upload_time = datetime.strptime(str(frame['upload_date'][i]), '%Y%m%d').timestamp()
        ms_since_upload = current_time - upload_time
        days_since_upload = ms_since_upload / 86400
        
        frame['upload_date'][i] =  days_since_upload
    except:
        pass

### Inspect data

In [ ]:
frame.head()

In [ ]:
len(frame.id.unique())

In [ ]:
frame.describe()

In [ ]:
print(df.columns.tolist())

In [ ]:
len(frame['categories'].apply(sorted).transform(tuple).unique())

In [ ]:
categories = pd.DataFrame(frame['categories'].apply(sorted).transform(tuple).unique())

In [ ]:
categories

In [ ]:
grouped = frame.groupby("categories")

In [ ]:
newFrame = frame['categories'].apply(sorted).transform(tuple)

In [ ]:
frame["tupleCat"] = newFrame

In [ ]:
frame["tupleCat"]

In [ ]:
frame.groupby("tupleCat").agg("count")["id"]

Note: make a bar chart of category video count

## Plotting relations between categories and other characteristics

### Upload date

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
frame.explode('categories').boxplot(by='categories', column=['upload_date'], ax=ax, grid=False)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.get_yaxis().set_ticks([365, 730, 1095, 1460, 1825])

### Popularity

#### Plot popularity

Note: like/dislike ratio is not being used for calculating the popularity of a video (see paragraph under graphs).

In [ ]:
# For each entry, calculate its popularity
frame['popularity'] = 0.0 # initial float value
frame['like_dislike_ratio'] = 0.0

for i in frame.index:
    view_count = frame['view_count'][i]
    days_since_upload = frame['upload_date'][i]
    like_count = frame['like_count'][i]
    dislike_count = frame['dislike_count'][i]

    like_dislike_ratio = like_count / dislike_count

    popularity = (view_count / float(days_since_upload))# * like_dislike_ratio

    frame['like_dislike_ratio'][i] = like_dislike_ratio
    frame['popularity'][i] =  popularity

In [ ]:
# Plot popularity

fig, ax = plt.subplots(figsize=(20, 10))
frame.explode('categories').boxplot(by='categories', column=['popularity'], ax=ax, grid=False, showfliers=False)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')

#### Plot like/dislike ratio

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
frame.explode('categories').boxplot(by='categories', column=['like_dislike_ratio'], ax=ax, grid=False, showfliers=False)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.get_yaxis().set_ticks([1, 10])

In the boxplot above, we find that the ratio between likes and dislikes on a video is hardly ever below 1. Therefore, we may conclude that viewers are more inclined to indicate which videos they like than to indicate which videos they dislike. And indeed, content creators usually encourage their audience to like their videos. Therefore, the ratio between likes and dislikes seems to be positively dominated by a relatively high number of likes.

This means that the like/dislike ratio may overrepresent the positive perception and we must be careful with using this metric.

### Available representations

#### Number of representations

Not solved yet.

In [ ]:
og_frame

In [ ]:
df1 = og_frame.copy()
df1['available_representations'] = df1['formats'].str.len()

for row in df1:
    for item in df1[row]:
        print(row)
        break
#         if df1[col][row].vcodec == "none":
#             df1.drop(row, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
df1.explode('categories').boxplot(by='categories', column=['available_representations'], ax=ax, grid=False, showfliers=False)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')

In [ ]:
# df1 = (pd.concat({i: pd.DataFrame(x) for i, x in frame.pop('formats').items()})
#          .reset_index(level=1, drop=True)
#          .join(frame, rsuffix='_shared')
#          .reset_index(drop=True))

# df1 = df1[df1.vcodec != "none

og_frame

#### Average bitrate

In [ ]:
# Create column for average bitrate (kbps)
df1['average_bitrate'] = 0.0

for i in df1.index:
    try:
        file_size_bytes = df1['filesize'][i]
        file_size_bits = file_size_bytes * 8
        
        duration = df1['duration'][i]
        
        average_bitrate = (file_size_bits / float(duration)) / 1000 # average bitrate in kbps
        
        df1['average_bitrate'][i] =  average_bitrate
    except:
        pass

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
df1.explode('categories').boxplot(by='categories', column=['average_bitrate'], ax=ax, grid=False, showfliers=False)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')

#### File types

In [ ]:
df1.groupby('categories')

In [ ]:
df1.groupby('ext').agg('count')["id"]

#### Framerates

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
df1.explode('categories').boxplot(by='categories', column=['fps'], ax=ax, grid=False, showfliers=True)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')

#### Durations

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
df1.explode('categories').boxplot(by='categories', column=['duration'], ax=ax, grid=False, showfliers=False)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')